# Лабораторна робота №3
## Вступ до опрацювання природної мови
__Виконали студенти групи ІА-01: Гнітій Є.О., Дронь О.В., Телесненко І.М.__


In [142]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

### 1) Завантажити англійську книгу «Alice's Adventures in Wonderland»

In [143]:
with open("book.txt", "r", encoding='utf-8') as file:
    text = file.read()
print(text)

﻿The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: Alice’s Adventures in Wonderland

Author: Lewis Carroll

Release Date: January, 1991 [eBook #11]
[Most recently updated: October 12, 2020]

Language: English

Character set encoding: UTF-8

Produced by: Arthur DiBianca and David Widger

*** START OF THE PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***

[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I.     Down the Rabbit-

### 2) Реалізувати пайплайн опрацювання обраного тексту англійською мовою, включно з усією необхідною попередньою обробкою тексту, включно з приведенням слів до нижнього регістру, видаленням стоп-слів, цифр/неалфавітних символів, знаків пунктуації.

In [144]:
import nltk
import re
from nltk.stem import WordNetLemmatizer

# Ініціалізація токенізатора, списку стоп-слів і лематизатора
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preproc_doc(doc):
    # Видалення розділових знаків та цифр
    doc = ''.join(e for e in doc if e.isalpha() or e.isspace())
    
    # Приведення тексту до нижнього регістру і видалення зайвих пробілів на початку і в кінці
    doc = doc.lower().strip()                                         
    
    # Токенізація тексту
    tokens = wpt.tokenize(doc)
    
    # Видалення стоп-слів
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Лематизація
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Повертаємо оброблений текст
    return ' '.join(lemmatized_tokens)

p_corpus = preproc_doc(text)  
print(p_corpus)

project gutenberg ebook alices adventure wonderland lewis carroll ebook use anyone anywhere united state part world cost almost restriction whatsoever may copy give away reuse term project gutenberg license included ebook online wwwgutenbergorg located united state check law country located using ebook title alices adventure wonderland author lewis carroll release date january ebook recently updated october language english character set encoding utf produced arthur dibianca david widger start project gutenberg ebook alices adventure wonderland illustration alices adventure wonderland lewis carroll millennium fulcrum edition content chapter rabbithole chapter ii pool tear chapter iii caucusrace long tale chapter iv rabbit sends little bill chapter v advice caterpillar chapter vi pig pepper chapter vii mad teaparty chapter viii queen croquetground chapter ix mock turtle story chapter x lobster quadrille chapter xi stole tart chapter xii alices evidence chapter rabbithole alice beginning

### 3) Розділити текст на глави і в кожній главі відібрати Топ-20 слів за допомогою алгоритму TF-IDF.

In [145]:
# Знаходження всіх позицій, де зустрічається "CHAPTER I"
positions = [match.start() for match in re.finditer(r'\bCHAPTER I\b', text)]

if len(positions) >= 2:
    # Відсікаємо текст до другого згадування "CHAPTER I"
    text = text[positions[1]:]
else:
    print("Друге згадування 'CHAPTER I' не знайдено!")

In [146]:
text

'CHAPTER I.\nDown the Rabbit-Hole\n\n\nAlice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into\nthe book her sister was reading, but it had no pictures or\nconversations in it, “and what is the use of a book,” thought Alice\n“without pictures or conversations?”\n\nSo she was considering in her own mind (as well as she could, for the\nhot day made her feel very sleepy and stupid), whether the pleasure of\nmaking a daisy-chain would be worth the trouble of getting up and\npicking the daisies, when suddenly a White Rabbit with pink eyes ran\nclose by her.\n\nThere was nothing so _very_ remarkable in that; nor did Alice think it\nso _very_ much out of the way to hear the Rabbit say to itself, “Oh\ndear! Oh dear! I shall be late!” (when she thought it over afterwards,\nit occurred to her that she ought to have wondered at this, but at the\ntime it all seemed quite natural); but when the Rabbit actually _took

In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Розділіть текст на глави за допомогою римських чисел
chapters = re.split(r'\bCHAPTER [IVXLCDM]+\b', text)

# Видалення першої глави, яка повертається пустою
chapters = chapters[1:]

# Виведення вмісту кожної глави
for idx, chapter in enumerate(chapters):
    print(f"Chapter {idx+1} content:\n")
    print(chapter)
    print("\n" + "-"*50 + "\n")

# Визначення довжини chapters
num_chapters = len(chapters)
print(f"Number of chapters: {num_chapters}")

Chapter 1 content:

.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so _very_ remarkable in that; nor did Alice think it
so _very_ much out of the way to hear the Rabbit say to itself, “Oh
dear! Oh dear! I shall be late!” (when she thought it over afterwards,
it occurred to her that she ought to have wondered at this, but at the
time it all seemed quite natural); but when the Rabbit actually _took a
watch o

In [148]:
# Обробка кожної глави
processed_chapters = [preproc_doc(chapter) for chapter in chapters]

# Застосування TF-IDF
vectorizer = TfidfVectorizer(max_df=0.85, max_features=20) # max_df відфільтровує слова, які з'являються занадто часто
tfidf_matrix = vectorizer.fit_transform(processed_chapters)

In [149]:
# Вибір Топ-20 слів для кожної глави
top_words = {}

feature_names = vectorizer.get_feature_names_out()
for idx, row in enumerate(tfidf_matrix):
    top_indices = row.toarray()[0].argsort()[-20:][::-1]
    top_features = [feature_names[i] for i in top_indices]
    top_words[f"Chapter {idx+1}"] = top_features

print(top_words)

{'Chapter 1': ['rabbit', 'cat', 'found', 'oh', 'right', 'mouse', 'work', 'im', 'project', 'queen', 'tone', 'voice', 'king', 'hatter', 'gutenbergtm', 'gryphon', 'turtle', 'duchess', 'dormouse', 'mock'], 'Chapter 2': ['mouse', 'im', 'cat', 'oh', 'rabbit', 'voice', 'right', 'duchess', 'tone', 'found', 'gutenbergtm', 'dormouse', 'gryphon', 'work', 'hatter', 'king', 'project', 'queen', 'turtle', 'mock'], 'Chapter 3': ['mouse', 'found', 'cat', 'tone', 'oh', 'voice', 'im', 'dormouse', 'duchess', 'gryphon', 'gutenbergtm', 'hatter', 'work', 'king', 'project', 'queen', 'rabbit', 'right', 'turtle', 'mock'], 'Chapter 4': ['rabbit', 'voice', 'oh', 'im', 'found', 'mouse', 'tone', 'right', 'duchess', 'work', 'gutenbergtm', 'dormouse', 'gryphon', 'mock', 'hatter', 'king', 'project', 'queen', 'turtle', 'cat'], 'Chapter 5': ['im', 'tone', 'right', 'found', 'voice', 'work', 'oh', 'dormouse', 'duchess', 'gryphon', 'gutenbergtm', 'hatter', 'mock', 'king', 'mouse', 'project', 'queen', 'rabbit', 'turtle', 'c

### 4) Реалізувати LDA алгоритм і порівняти результати з отриманими раніше за допомогою TF-IDF. Зробити висновки про застосовність реалізованих підходів.

In [150]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 1. Використання CountVectorizer
vectorizer = CountVectorizer(max_df=0.90, min_df=2)
dtm = vectorizer.fit_transform(processed_chapters)

# 2. LDA
lda_model = LatentDirichletAllocation(n_components=num_chapters, random_state=42)
lda_model.fit(dtm)

# Функція для виведення тем з LDA
def print_lda_topics(model, vectorizer, n_words=20):
    words = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(model.components_):
        print(f"Chapter {idx+1}:", [words[i] for i in topic.argsort()[-n_words:][::-1]])
#         print("\n")

# 3. Виведення тем
print_lda_topics(lda_model, vectorizer)

Chapter 1: ['work', 'state', 'king', 'term', 'law', 'may', 'full', 'use', 'set', 'jury', 'queen', 'agree', 'without', 'white', 'rabbit', 'gave', 'day', 'sister', 'charge', 'please']
Chapter 2: ['caterpillar', 'im', 'size', 'bit', 'youre', 'tried', 'last', 'mouth', 'yet', 'felt', 'hand', 'side', 'right', 'old', 'tone', 'good', 'found', 'day', 'replied', 'father']
Chapter 3: ['rabbit', 'bill', 'heard', 'sure', 'made', 'room', 'moment', 'voice', 'ran', 'fan', 'glove', 'bottle', 'door', 'till', 'put', 'oh', 'house', 'arm', 'grow', 'next']
Chapter 4: ['cat', 'duchess', 'mad', 'baby', 'pig', 'door', 'cook', 'felt', 'right', 'oh', 'im', 'sitting', 'youre', 'day', 'house', 'tone', 'grin', 'march', 'child', 'creature']
Chapter 5: ['turtle', 'mock', 'gryphon', 'duchess', 'voice', 'soup', 'queen', 'beautiful', 'tone', 'wont', 'replied', 'sea', 'course', 'heard', 'day', 'old', 'sat', 'far', 'take', 'last']
Chapter 6: ['door', 'found', 'eat', 'either', 'going', 'wonder', 'key', 'rabbit', 'bat', 'su

In [151]:
# 4. Порівняння з TF-IDF
print(top_words)

{'Chapter 1': ['rabbit', 'cat', 'found', 'oh', 'right', 'mouse', 'work', 'im', 'project', 'queen', 'tone', 'voice', 'king', 'hatter', 'gutenbergtm', 'gryphon', 'turtle', 'duchess', 'dormouse', 'mock'], 'Chapter 2': ['mouse', 'im', 'cat', 'oh', 'rabbit', 'voice', 'right', 'duchess', 'tone', 'found', 'gutenbergtm', 'dormouse', 'gryphon', 'work', 'hatter', 'king', 'project', 'queen', 'turtle', 'mock'], 'Chapter 3': ['mouse', 'found', 'cat', 'tone', 'oh', 'voice', 'im', 'dormouse', 'duchess', 'gryphon', 'gutenbergtm', 'hatter', 'work', 'king', 'project', 'queen', 'rabbit', 'right', 'turtle', 'mock'], 'Chapter 4': ['rabbit', 'voice', 'oh', 'im', 'found', 'mouse', 'tone', 'right', 'duchess', 'work', 'gutenbergtm', 'dormouse', 'gryphon', 'mock', 'hatter', 'king', 'project', 'queen', 'turtle', 'cat'], 'Chapter 5': ['im', 'tone', 'right', 'found', 'voice', 'work', 'oh', 'dormouse', 'duchess', 'gryphon', 'gutenbergtm', 'hatter', 'mock', 'king', 'mouse', 'project', 'queen', 'rabbit', 'turtle', 'c